# Market segmentation insurance
## ETL process

## 1. Introduction

The purpose of this notebook is to create ETL process, which takes raw data and transform it to useful data.

## 2. Data information

This dataset summarizes the usage behavior of about 9000 active credit cardholders during the last 6 months.

Features:

* Cust ID
* Balance
* Balance Frequency
* Purchases
* One-off Purchases
* Installment Purchases
* Cash Advance
* Purchases Frequency
* One-off Purchases Frequency
* Purchases Installments Frequency
* Cash Advance Frequency
* Cash Advance TRX
* Purchases TRX
* Credit Limit
* Payments
* Minimum Payments
* PRC Full payment
* Tenure

Link: https://www.kaggle.com/datasets/jillanisofttech/market-segmentation-in-insurance-unsupervised/data

## 3. ETL process

Set catalog and schema

In [0]:
%sql
USE CATALOG training_catalog;

In [0]:

%sql
USE SCHEMA market_segmentation_insurance_db;

In [0]:
%sql
SHOW TABLES;

### 3.1. Extract

This step saves raw data into a raw_data delta table. This step also performs BRONZE part of ETL process.

Verify data

In [0]:
%sql
SELECT *
FROM read_files(
  "/Volumes/training_catalog/market_segmentation_insurance_db/training_files/",
  format => "csv",
  header => true,
  inferSchema => true
) LIMIT 5;

Load data into raw_data table

In [0]:
spark.sql('''
          COPY INTO raw_data
          FROM "/Volumes/training_catalog/market_segmentation_insurance_db/training_files/"
          FILEFORMAT = CSV
          FORMAT_OPTIONS (
            "header" = "true",
            "inferSchema" = "false" -- Use schema defined during table creation
          )
          ''').display()

In [0]:
%sql
SELECT * FROM raw_data LIMIT 5;

In [0]:
%sql
SELECT COUNT(*) FROM raw_data;

### 3.2. Transform

This process focuses on validating and transforming data in order to get it done to be used for other process. At the end, the goal is to create SILVER and GOLD tables.

#### 3.2.1. Data profiling

##### 3.2.1.1. Identify missing values

In [0]:
%sql
SELECT column_name, missing_count
FROM (
  SELECT
    COUNT(CASE WHEN balance IS NULL OR TRIM(balance) = '' THEN 1 END) AS missing_balance,
    COUNT(CASE WHEN balance_frequency IS NULL OR TRIM(balance_frequency) = '' THEN 1 END) AS missing_balance_frequency,
    COUNT(CASE WHEN purchases IS NULL OR TRIM(purchases) = '' THEN 1 END) AS missing_purchases,
    COUNT(CASE WHEN oneoff_purchases IS NULL OR TRIM(oneoff_purchases) = '' THEN 1 END) AS missing_oneoff_purchases,
    COUNT(CASE WHEN installments_purchases IS NULL OR TRIM(installments_purchases) = '' THEN 1 END) AS missing_installments_purchases,
    COUNT(CASE WHEN cash_advance IS NULL OR TRIM(cash_advance) = '' THEN 1 END) AS missing_cash_advance,
    COUNT(CASE WHEN purchases_frequency IS NULL OR TRIM(purchases_frequency) = '' THEN 1 END) AS missing_purchases_frequency,
    COUNT(CASE WHEN oneoff_purchases_frequency IS NULL OR TRIM(oneoff_purchases_frequency) = '' THEN 1 END) AS missing_oneoff_purchases_frequency,
    COUNT(CASE WHEN purchases_installments_frequency IS NULL OR TRIM(purchases_installments_frequency) = '' THEN 1 END) AS missing_purchases_installments_frequency,
    COUNT(CASE WHEN cash_advance_frequency IS NULL OR TRIM(cash_advance_frequency) = '' THEN 1 END) AS missing_cash_advance_frequency,
    COUNT(CASE WHEN cash_advance_trx IS NULL OR TRIM(cash_advance_trx) = '' THEN 1 END) AS missing_cash_advance_trx,
    COUNT(CASE WHEN purchases_trx IS NULL OR TRIM(purchases_trx) = '' THEN 1 END) AS missing_purchases_trx,
    COUNT(CASE WHEN credit_limit IS NULL OR TRIM(credit_limit) = '' THEN 1 END) AS missing_credit_limit,
    COUNT(CASE WHEN payments IS NULL OR TRIM(payments) = '' THEN 1 END) AS missing_payments,
    COUNT(CASE WHEN minimum_payments IS NULL OR TRIM(minimum_payments) = '' THEN 1 END) AS missing_minimum_payments,
    COUNT(CASE WHEN prc_full_payment IS NULL OR TRIM(prc_full_payment) = '' THEN 1 END) AS missing_prc_full_payment,
    COUNT(CASE WHEN tenure IS NULL OR TRIM(tenure) = '' THEN 1 END) AS missing_tenure
  FROM raw_data
) t
LATERAL VIEW STACK(17,
  'balance', missing_balance,
  'balance_frequency', missing_balance_frequency,
  'purchases', missing_purchases,
  'oneoff_purchases', missing_oneoff_purchases,
  'installments_purchases', missing_installments_purchases,
  'cash_advance', missing_cash_advance,
  'purchases_frequency', missing_purchases_frequency,
  'oneoff_purchases_frequency', missing_oneoff_purchases_frequency,
  'purchases_installments_frequency', missing_purchases_installments_frequency,
  'cash_advance_frequency', missing_cash_advance_frequency,
  'cash_advance_trx', missing_cash_advance_trx,
  'purchases_trx', missing_purchases_trx,
  'credit_limit', missing_credit_limit,
  'payments', missing_payments,
  'minimum_payments', missing_minimum_payments,
  'prc_full_payment', missing_prc_full_payment,
  'tenure', missing_tenure
) AS column_name, missing_count;

**Conclusion:**

* From all variables, only **credit_limit** and **minimum_payments** have missing values. Variable **credit_limit** has 1 and variable **minimum_payments** has 313.

##### 3.2.1.2. Identify duplicate values

In [0]:
%sql
SELECT
  balance,
  balance_frequency,
  purchases,
  oneoff_purchases,
  installments_purchases,
  cash_advance,
  purchases_frequency,
  oneoff_purchases_frequency,
  purchases_installments_frequency,
  cash_advance_frequency,
  cash_advance_trx,
  purchases_trx,
  credit_limit,
  payments,
  minimum_payments,
  prc_full_payment,
  tenure,
  COUNT(*) count_duplicated
FROM raw_data
GROUP BY
  balance,
  balance_frequency,
  purchases,
  oneoff_purchases,
  installments_purchases,
  cash_advance,
  purchases_frequency,
  oneoff_purchases_frequency,
  purchases_installments_frequency,
  cash_advance_frequency,
  cash_advance_trx,
  purchases_trx,
  credit_limit,
  payments,
  minimum_payments,
  prc_full_payment,
  tenure
HAVING COUNT(*) > 1;

**Conclusion:**

* There are no duplicated values.

##### 3.2.1.3. Validate data consistency

Several columns which could store numeric values were detected, so before applying any numeric transformation, it is important to verify if values are numeric.

In [0]:
%sql
SELECT
  'balance' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(balance, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'balance_frequency' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(balance_frequency, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'purchases' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(purchases, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'oneoff_purchases' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(oneoff_purchases, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'installments_purchases' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(installments_purchases, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'cash_advance' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(cash_advance, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'purchases_frequency' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(purchases_frequency, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'oneoff_purchases_frequency' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(oneoff_purchases_frequency, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'purchases_installments_frequency' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(purchases_installments_frequency, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'cash_advance_frequency' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(cash_advance_frequency, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'cash_advance_trx' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(cash_advance_trx, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'purchases_trx' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(purchases_trx, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'credit_limit' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(credit_limit, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'minimum_payments' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(minimum_payments, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

In [0]:
%sql
SELECT
  'prc_full_payment' AS column_name,
  (SELECT COUNT(1) FROM raw_data WHERE NOT REGEXP(prc_full_payment, '^[0-9]+(\\.[0-9]+)?$')) AS count_invalid;

**Conclusion:**

* All numeric features have no invalid values.

#### 3.2.2. Cleaning

To perform this process, it is possible to do many tasks in one step (creating view).

Tasks:

* Update datatype.
* Filling missing values.
* Remove white spaces.

Create view to show median values.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW data_median_view AS
SELECT 
  percentile_approx(CAST(credit_limit AS DOUBLE), 0.5) AS credit_limit_median,
  percentile_approx(CAST(minimum_payments AS DOUBLE), 0.5) AS minimum_payments_median
FROM raw_data;

In [0]:
%sql
SELECT * FROM data_median_view;

Create a view to store cleaned data.

Tasks:

* All features (except **tenure**) are transformed to DOUBLE.
* Variable **tenure** is applied by TRIM funcion to remove white spaces.
* Variables **credit_limit** and **minimum_payments** tackle missing-values problem by complete them with the MEDIAN.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW data_cleaned_view AS
SELECT
  CAST(balance AS DOUBLE) AS balance,
  CAST(balance_frequency AS DOUBLE) AS balance_frequency,
  CAST(purchases AS DOUBLE) AS purchases,
  CAST(oneoff_purchases AS DOUBLE) AS oneoff_purchases,
  CAST(installments_purchases AS DOUBLE) AS installments_purchases,
  CAST(cash_advance AS DOUBLE) AS cash_advance,
  CAST(purchases_frequency AS DOUBLE) AS purchases_frequency,
  CAST(oneoff_purchases_frequency AS DOUBLE) AS oneoff_purchases_frequency,
  CAST(purchases_installments_frequency AS DOUBLE) AS purchases_installments_frequency,
  CAST(cash_advance_frequency AS DOUBLE) AS cash_advance_frequency,
  CAST(cash_advance_trx AS DOUBLE) AS cash_advance_trx,
  CAST(purchases_trx AS DOUBLE) AS purchases_trx,
  IFNULL(CAST(credit_limit AS DOUBLE), (SELECT credit_limit_median FROM data_median_view)) AS credit_limit,
  CAST(payments AS DOUBLE) AS payments,
  IFNULL(CAST(minimum_payments AS DOUBLE), (SELECT minimum_payments_median FROM data_median_view)) AS minimum_payments,
  CAST(prc_full_payment AS DOUBLE) AS prc_full_payment,
  TRIM(tenure) AS tenure
FROM raw_data;

Verify if there is still missing values

In [0]:
%sql
SELECT * FROM data_cleaned_view WHERE credit_limit IS NULL LIMIT 5;

In [0]:
%sql
SELECT * FROM data_cleaned_view WHERE minimum_payments IS NULL LIMIT 5;

Show a sample of cleaned data.

In [0]:
%sql
SELECT * FROM data_cleaned_view LIMIT 5;

### 3.3. Load

Load cleaned data in a SILVER table.

In [0]:
%sql
CREATE OR REPLACE TABLE silver_data AS
SELECT * FROM data_cleaned_view;

In [0]:
%sql
SELECT * FROM silver_data LIMIT 5;

Drop views

In [0]:
%sql
DROP VIEW IF EXISTS data_cleaned_view;
DROP VIEW IF EXISTS data_median_view;